In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score
from kneed import KneeLocator
from scipy import stats  # Import the stats module for statistical methods
from scipy.stats import shapiro, levene, kruskal
import scikit_posthocs as sp

print("Successful Import!")

In [ ]:
# Replace 'file_path' with the path to your data file
file_path = r"C:\Users\Farhan\Videos\3D Mitochondrial Analysis\Test Data.xlsx"
df = pd.read_excel(file_path) # Note: change pd.read_excel to pd.read_csv if you want to use a CSV file

# Display the first few rows of the DataFrame
print("First few rows of the data:")
display(df.head())

# Group by 'Image Name' and 'Cell Type', and calculate the number of unique 'Image Name' values in each group

In [1]:
unique_image_counts = df.groupby(['Image Name', 'Cell Type']).nunique()

print("\nNote: Ensure that the image names are not repeating")
# Display the grouped DataFrame
print("Unique image counts per cell type:")
display(unique_image_counts)

# Group by 'Image Name' and 'Cell Type', and calculate the descriptive statistics for each group
grouped = df.groupby(['Image Name', 'Cell Type']).describe()

print("\nDescriptive Statistics:")
# Display the grouped DataFrame
display(grouped)

# Display Box Plots of Grouped Data (Change Numeric Column Names Based on Your Own DataSet)_

In [ ]:
# List of numeric columns.
numeric_columns = ['Count', 'Total Volume', 'Mean Volume', 'Total Surface Area', 'Mean Surface Area', 'Sphericity (Weighted)', 'Branches', 'Branches/volume', 'Total Branch Length', 'Total Branch Length/volume', 'Mean Branch Length', 'Branch Junctions', 'Branch Junctions/volume', 'Branch End Points', 'Branch End Points/volume', 'Mean Branch Diameter']

# Create a figure and axes objects
fig, axes = plt.subplots(len(numeric_columns), 1, figsize=(10, len(numeric_columns) * 5))

# For each numeric column
for i, col in enumerate(numeric_columns):
    # Create a box plot for the overall data
    axes[i].boxplot(df[col].dropna(), positions=[1], widths=0.5)

    # For each cell type
    for j, cell_type in enumerate(df['Cell Type'].unique(), start=2):
        # Create a box plot for the cell type specific data
        axes[i].boxplot(df[df['Cell Type'] == cell_type][col].dropna(), positions=[j], widths=0.2)

    # Set the x-ticks and x-tick labels
    axes[i].set_xticks(range(1, len(df['Cell Type'].unique()) + 2))
    axes[i].set_xticklabels(['Overall'] + list(df['Cell Type'].unique()))

    # Set the title
    axes[i].set_title(f'Box plot of {col}')

    # Set the y-label
    axes[i].set_ylabel(col)

# Display the plot
plt.tight_layout()
plt.show()

# Perform Correlation Analysis, Print Correlation Matrices, and Plot the Correlation Data.

In [ ]:
def compute_correlations(df, method='spearman'): # Change the type of correlation you want to do in this line!
    # Choose the correlation method
    if method == 'pearson':
        corr_func = stats.pearsonr
    elif method == 'spearman':
        corr_func = stats.spearmanr
    else:
        raise ValueError("method must be either 'pearson' or 'spearman'")

    # Get the list of numeric columns
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

    # Initialize a DataFrame to hold the correlation coefficients and p-values
    corr_df = pd.DataFrame(index=numeric_cols, columns=numeric_cols)
    pval_df = pd.DataFrame(index=numeric_cols, columns=numeric_cols)

    # Compute the correlation for each pair of columns
    for i, col1 in enumerate(numeric_cols):
        for j, col2 in enumerate(numeric_cols[i:]):
           # Calculate correlation using the specified method, handling NaNs by dropping them
            corr, pval = corr_func(df[col1].dropna(), df[col2].dropna())
            corr_df.loc[col1, col2] = corr
            pval_df.loc[col1, col2] = pval

    # Apply Bonferroni correction to the p-values
    num_comparisons = len(numeric_cols) * (len(numeric_cols) - 1) // 2 # Calculate number of pairs
    pval_df = pval_df * num_comparisons

    return corr_df, pval_df

# Compute the correlations
corr_df, pval_df = compute_correlations(df, method='spearman')  # or method='spearman'

# Create a copy of the correlation matrix
marked_corr_df = corr_df.copy()

# Mark the correlation coefficients with significant p-values
marked_corr_df[pval_df < 0.05] = marked_corr_df[pval_df < 0.05].astype(str) + '**'

# Display the correlation coefficients as a table
print("\nTable 1: Correlation Coefficients")
display(marked_corr_df)
print("** = P-value < 0.05")

print("--------------------------------------------")

# Display the p-values as a table
#print("Table 2: P-values for Correlation Coefficients")
print("\nP-values for Correlation Coefficients:")
display(pval_df)

#print("\nNote: The 'NaN' values in the table could be due to zeros or missing data in the corresponding columns.")

# Compute the correlations
corr_df, pval_df = compute_correlations(df, method='spearman')  # or method='spearman'

# Replace NaN values with 0
corr_df.fillna(0, inplace=True)
corr_df = corr_df.infer_objects(copy=False)
# Create a heatmap of the correlation matrix
plt.figure(figsize=(5, 5))
sns.heatmap(corr_df, annot=False, cmap='coolwarm')
plt.title('Correlation Heatmap')
#plt.savefig(r"C:\3D Mitochondrial Analysis\PerCell_Original_data_heatmap_10-16-24.png", bbox_inches='tight', dpi=800)
plt.show()

# Compute the correlations
corr_df, pval_df = compute_correlations(df, method='spearman')  # or method='spearman'

# Replace NaN values with 0
corr_df.fillna(0, inplace=True)
corr_df = corr_df.infer_objects(copy=False)
# Create a heatmap of the correlation matrix
fig, ax = plt.subplots(figsize=(5, 5))  # Adjust as necessary
sns.heatmap(corr_df, annot=False, cmap='coolwarm', ax=ax) # 'coolwarm' also works great
plt.title('Correlation Heatmap (Per Cell) Control and FA Conditions')

# Perform PCA Analysis 

In [ ]:
# Exclude non-numeric columns from df
df_numeric = df.select_dtypes(include=[np.number])

# Initialize PCA with 2 components
pca = PCA(n_components=2)

# Fit and transform df_numeric, not df
reduced_df = pca.fit_transform(df_numeric)

# Plot the reduced data
plt.scatter(reduced_df[:, 0], reduced_df[:, 1])
plt.xlabel('Principal Component 1', fontsize=12)
plt.ylabel('Principal Component 2', fontsize=12)
plt.title('PCA Scatter Plot (Per Cell)', fontsize=16)
#plt.savefig(r"C:\Videos\3D Mitochondrial Analysis\PerCell_PCA_Scatter_Plot_10-16-24.png", bbox_inches='tight', dpi=800)

plt.show()


# Identify optimal Cluster Count for your Own Data.

In [ ]:
# Exclude non-numeric columns from df
df_numeric = df.select_dtypes(include=[np.number])

distortions = []
K = range(1, 10)
for k in K:
    kmeanModel = KMeans(n_clusters=k, n_init=10, random_state=42).fit(df_numeric)  # Added n_init and random_state
    distortions.append(sum(np.min(cdist(df_numeric, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / df_numeric.shape[0])

# Use KneeLocator to find the elbow point
kl = KneeLocator(range(1, 10), distortions, curve="convex", direction="decreasing")

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal "k" for clustering', fontsize=16)

# Annotate the optimal k
plt.annotate('Elbow point', xy=(kl.elbow, distortions[kl.elbow - 1]),
             xytext=(kl.elbow, distortions[kl.elbow - 1] + 0.05),
             arrowprops=dict(facecolor='black', shrink=0.05))

# Add a red dot to mark the elbow point
plt.plot(kl.elbow, distortions[kl.elbow - 1], 'ro')

# Save and show the plot
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal "k" for clustering', fontsize=16)
#plt.savefig(r"C:\Videos\3D Mitochondrial Analysis\PerCell_Elbow_K_Plot_10-16-24.png", bbox_inches='tight', dpi=800)

# Show the plot
plt.show()

# Use the Optimal Cluster Count from the Elbow Method below (Change the `n_clusters` to whatever you get from the elbow method)

In [ ]:
kmeans = KMeans(n_clusters=4, n_init=10, random_state=42)  # Choose the appropriate number of clusters, added random_state
kmeans.fit(reduced_df)

# The cluster labels are in kmeans.labels_
df['cluster'] = kmeans.labels_

# You can now examine the features of the mitochondria in each cluster
for cluster in set(kmeans.labels_):
    print(f"\n Features of Mitochondria in Cluster {cluster}:")
    display(df[df['cluster'] == cluster].describe())


# 'labels' is an array of cluster labels for each data point
# 'df' is original dataset
score = silhouette_score(reduced_df, kmeans.labels_)

# Create a scatter plot of the reduced data
plt.figure(figsize=(10, 8))
plt.scatter(reduced_df[:, 0], reduced_df[:, 1], c=kmeans.labels_, cmap='viridis')

# If your clustering algorithm provides cluster centers (like KMeans), you can plot them as well
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='red')

plt.xlabel('Principal Component 1', fontsize=12)
plt.ylabel('Principal Component 2', fontsize=12)
plt.title('PCA Scatter Plot with K-Mean Clusters', fontsize=16)
#plt.savefig(r"C:\Users\Farhan\Videos\3D Mitochondrial Analysis\PerCell_K-Mean_Clusters_Plot_10-16-24.png", bbox_inches='tight', dpi=800) # Save K-mean clustering PNG to your folder
plt.show()

#print("Silhouette Score: ", score)
print("\nNote: The Silhouette Score is a single measure that provides a summary of the compactness and separation of the clusters in your data. It’s calculated using all the data points and their corresponding cluster assignments. The Silhouette Score is a measure of how similar an object is to its own cluster compared to other clusters. It ranges from -1 to 1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.")

# Save the entire dataframe with cluster labels to a CSV file
#df.to_csv(r"D:\Mitochondria Morphology Analysis\clustered_PerCell_data_07-17-24.csv", index=False)

# Assume that 'df' is your DataFrame and 'cluster' is the column with cluster labels
for col in df_numeric.columns:
    if col != 'cluster':
        sns.boxplot(x='cluster', y=col, data=df)
        plt.title(f'Box plot of {col} by cluster')

        # Save the plot as a PNG file
        #plt.savefig(f"D:\\Mitochondria Morphology Analysis\\PerCell_ClusteredBoxPlots__07-17-24_{col}.png") # use "\\" Double slashes for paths in this line

        plt.show()

# Group Data based on Clustering and Display Graphs for each paramter

In [ ]:
# Exclude non-numeric columns from df
df_numeric = df.select_dtypes(include=[np.number])

# Calculate the mean values
mean_values = df_numeric.groupby('cluster').mean()
#mean_values.to_csv(r"D:\Mitochondria Morphology Analysis\PerCell_MeanMitoValuesByCluster_data_07-17-24.csv", index=False)

# Create a heatmap
fig, ax = plt.subplots(figsize=(10, 8))  # Adjust as necessary
sns.heatmap(mean_values, cmap='coolwarm', annot=False)
plt.title('Heatmap of mean mitochondrial parameter values by cluster')
#plt.savefig(r"D:\Mitochondria Morphology Analysis\PerCell_MeanMitoValuesByCluster_07-17-24.png", bbox_inches='tight',dpi=2400)
plt.show()
print("\nMean Values by Cluster:")
display(mean_values)
# Group by 'Cell Type' and 'cluster', and calculate the number of distinct images in each group
grouped = df.groupby(['Cell Type', 'cluster'])['Image Name'].nunique()

# Calculate the total number of distinct images in each cell type
total = df.groupby('Cell Type')['Image Name'].nunique()

# Divide the grouped data by the total data and multiply by 100 to get percentages
percentages = grouped / total * 100

print("\nTable _Percent of Cells in each Cluster:")
display(percentages)
print("-------------------------------------------------------------")
#print("Table _Total Number of Cells from each Cell Type in each Cluster:")
print("\nTotal Number of Cells from each Cell Type in each Cluster:")
display(grouped)

# Find the Most Common Cluster For Each Image/Cell (Assuming that each Image is One Cell)

In [ ]:
# Find the most common cluster for each image (or Cell, if you have cropped your images to represent one cell per Image)
image_clusters = df.groupby('Image Name')['cluster'].agg(lambda x: x.value_counts().index[0])

# Add the cell type information
image_clusters = df.drop_duplicates('Image Name').set_index('Image Name')['Cell Type'].map(str) + ' ' + image_clusters.map(str)

# Count the number of images of each cell type in each cluster
image_cluster_counts = image_clusters.value_counts()

# Convert the image_cluster_counts Series to a DataFrame and reset the index
image_cluster_counts_df = image_cluster_counts.reset_index()
image_cluster_counts_df.columns = ['Cell Type and Cluster', 'Count']

# Convert the 'Cell Type and Cluster' column to two separate columns
image_cluster_counts_df[['Cell Type', 'Cluster']] = image_cluster_counts_df['Cell Type and Cluster'].str.split(' ', expand=True)

# Pivot the DataFrame to get the counts for each cluster in separate columns
pivot_df = image_cluster_counts_df.pivot(index='Cell Type', columns='Cluster', values='Count').fillna(0)

# Create a stacked bar plot
#pivot_df.plot(kind='bar', stacked=True, figsize=(10, 6))

#plt.title('Count of Each Cell Type in Each Cluster', fontsize=16)
#plt.xlabel('Cell Type', fontsize=12)
#plt.ylabel('Count', fontsize=12)
#plt.xticks(rotation=0)  # Rotate the x-axis labels for better readability
# plt.show()

plt.figure()
# Create a stacked bar plot
ax = pivot_df.plot(kind='bar', stacked=True, figsize=(10, 6))

# Add the cell counts as text
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy()
    ax.text(x + width / 2,
            y + height / 2,
            '{:.0f}'.format(height),
            horizontalalignment='center',
            verticalalignment='center')

plt.title('Count of Each Cell Type in Each Cluster', fontsize=16)
plt.xlabel('Cell Type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)  # Rotate the x-axis labels for better readability
#plt.savefig(r"D:\Mitochondria Morphology Analysis\PerCell_CellTypePerCluster_07-17-24.png", bbox_inches='tight', dpi=2400)
plt.show()

# Find the most common cluster for each image/cell under each exposure condition (Assuming that each Image is One Cell)

In [ ]:
exposure_clusters = df.groupby(['Exposure Condition', 'Image Name'])['cluster'].agg(lambda x: x.value_counts().index[0])

# Add the exposure condition information
exposure_clusters = df.drop_duplicates('Image Name').set_index('Image Name')['Exposure Condition'].map(str) + '_' + exposure_clusters.map(str)

# Count the number of images of each exposure condition in each cluster
exposure_cluster_counts = exposure_clusters.value_counts()

# Convert the exposure_cluster_counts Series to a DataFrame and reset the index
exposure_cluster_counts_df = exposure_cluster_counts.reset_index()
exposure_cluster_counts_df.columns = ['Exposure Condition and Cluster', 'Count']

# Convert the 'Exposure Condition and Cluster' column to two separate columns
exposure_cluster_counts_df[['Exposure Condition', 'Cluster']] = exposure_cluster_counts_df['Exposure Condition and Cluster'].str.split('_', expand=True)

# Pivot the DataFrame to get the counts for each cluster in separate columns
pivot_df = exposure_cluster_counts_df.pivot(index='Exposure Condition', columns='Cluster', values='Count').fillna(0)

# Create a stacked bar plot
plt.figure()
ax = pivot_df.plot(kind='bar', stacked=True, figsize=(10, 6))

# Add the image counts as text
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy()
    ax.text(x + width / 2,
            y + height / 2,
            '{:.0f}'.format(height),
            horizontalalignment='center',
            verticalalignment='center')

plt.title('Count of Each Exposure Condition in Each Cluster', fontsize=16)
plt.xlabel('Exposure Condition', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=90)  # Rotate the x-axis labels for better readability
#plt.savefig(r"D:\Mitochondria Morphology Analysis\PerCell_CellsPerClusterPerExposure_07-17-24.png", bbox_inches='tight', dpi=2400)
plt.show()

# Shapiro-Wilk test for normality & Levene's Test for Equality of Variance

In [ ]:
# Initialize an empty list to store the results
results = []

# For each numeric column
for col in numeric_columns:
    # Perform the Shapiro-Wilk test for normality
    sw_stat, sw_p_val = shapiro(df[col].dropna())
    sw_test = 'Non-Normal' if sw_p_val < 0.05 else 'Normal'

    # For Levene's test, we need at least two groups. Let's use 'Exposure Condition' for grouping
    groups = [group[col].dropna() for name, group in df.groupby('Exposure Condition')]

    # Perform Levene's test for equality of variances
    lev_stat, lev_p_val = levene(*groups)
    lev_test = 'Unequal Variances' if lev_p_val < 0.05 else 'Equal Variances'

    # Append the results to the list
    results.append(pd.DataFrame({'Mitochondrial Parameter': [col], 'Shapiro-Wilk Statistic': [sw_stat], 'Shapiro-Wilk P-value': [sw_p_val], 'Shapiro-Wilk Test': [sw_test], 'Levene Statistic': [lev_stat], 'Levene P-value': [lev_p_val], 'Levene Test': [lev_test]}))

# Concatenate the results into a single DataFrame
results_SWL_df = pd.concat(results, ignore_index=True)
#results_SWL_df.to_csv(r"D:\Julia-Fura_FF_Data_Analysis\Mitochondria Morphology Analysis\clustered_PerCell_Shapiro-wilk_Levene's Test_07-16-24.csv", index=False)
# Display the results
print("\nShapiro-Wilk and Levene's Test Results:")
display(results_SWL_df)


# Kuskal-Wallis Test (non-parametric one-way ANOVA that tests whether samples originat from the same distribution. It is used for comparing two or more independent samples of **equal or different sample sizes.**)

In [ ]:
# Initialize an empty list to store the results
results = []

# For each numeric column
for col in numeric_columns:
    # Split the data into groups based on 'Exposure Condition'
    groups = [group[col].dropna() for name, group in df.groupby('Exposure Condition')]

    # Perform the Kruskal-Wallis H-test
    h_stat, p_val = kruskal(*groups)

    # Interpret the result
    kw_test = 'Significant' if p_val < 0.05 else 'Not Significant'

    # Append the results to the list
    results.append(pd.DataFrame({'Mitochondrial Parameter': [col], 'Kruskal-Wallis H-statistic': [h_stat], 'Kruskal-Wallis P-value': [p_val], 'Kruskal-Wallis Test': [kw_test]}))

# Concatenate the results into a single DataFrame
results_KW_df = pd.concat(results, ignore_index=True)
#results_KW_df.to_csv(r"D:\Julia-Fura_FF_Data_Analysis\Mitochondria Morphology Analysis\clustered_PerCell_Kruskal-Wallis_Test_Mitochondrial_Parameter_Across_FA_07-16-24.csv", index=False)
# Print the title
print("\nTable 1: Kruskal-Wallis H-Test Results for Each Mitochondrial Parameter Across Exposure Conditions")

# Display the results
display(results_KW_df)

# Dunn's Post-Hoc Pair-wise comparision (Uses Bonferroni Correction for multiple comparision. Adjust to holm if less conservative correction is needed; adjust to Benjamini-Hochberg (BH) or False Discovery Rate (FDR) if more liberal correction of p-values is required). 

### **Holm Correction:** The Holm correction is less conservative than Bonferroni. It adjusts p-values by comparing them to decreasing thresholds based on the rank of their significance.
### **Benjamini-Hochberg (BH) or False Discovery Rate (FDR):** The Benjamini-Hochberg (BH) procedure controls the False Discovery Rate (FDR), which is the expected proportion of rejected null hypotheses that are actually true. It's more liberal (less conservative) than Bonferroni and Holm, making it a good choice when you have a large number of tests and are more interested in controlling the proportion of false positives rather than the overall probability of at least one false positive.
### **Benjamini-Yekutieli (BY):** The Benjamini-Yekutieli procedure is another FDR control method that accounts for dependence among the tests, making it more robust to deviations in the distribution of test statistics from independence.

### **Additional Notes:**
##### **Conservatism:** Bonferroni is the most conservative (least likely to find a significant result), followed by Holm, and then BH/FDR/BY. Which one is most appropriate depends on the context of your research and how you want to balance the risk of false positives and false negatives.

##### **Number of Tests:** When you have a large number of comparisons, Bonferroni can be overly strict, and you might want to consider a more liberal method like FDR or BY.

##### **Assumptions:** BH/FDR assumes the tests are independent or weakly dependent (although BY can account for dependency). If your tests are strongly dependent, you might need to consider other more specialized methods.

In [ ]:
# Initialize an empty list to store all results
all_results = []

# For each numeric column
for col in numeric_columns:
    # Perform Dunn's test
    results_dunn = sp.posthoc_dunn(df, val_col=col, group_col='Exposure Condition', p_adjust='bonferroni') #(Change 'bonferroni' to 'holm', 'fdr_bh', or 'fdr_by' based on your p-value correction needs)

    # Add a column for the mitochondrial parameter
    results_dunn['Mitochondrial Parameter'] = col

    # Append the results to all_results list
    all_results.append(results_dunn)

    # Print the results
    print(f"\nDunn's post-hoc test for {col}:")
    display(results_dunn)

# Concatenate all results into a single DataFrame
results_df = pd.concat(all_results)

# Save all results to a CSV file
#results_df.to_csv(r"D:\Julia-Fura_FF_Data_Analysis\Mitochondria Morphology Analysis\clustered_PerCell_Dunn_Test_w_Bonferroni_FA_07-16-24.csv", index=True)



# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# For each numeric column
for col in numeric_columns:
    # Perform Dunn's test
    results_dunn = sp.posthoc_dunn(df, val_col=col, group_col='Exposure Condition', p_adjust='bonferroni')

    # Add a level to the columns for the mitochondrial parameter
    results_dunn.columns = pd.MultiIndex.from_product([[col], results_dunn.columns])

    # Concatenate the results into the all_results DataFrame
    all_results = pd.concat([all_results, results_dunn], axis=1)

# Create a heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(all_results, cmap='coolwarm', annot=False, fmt=".2f")

plt.title('Heatmap of Pairwise Dunn\'s P-values for All Mitochondrial Parameters', fontsize=16, y=1.05)
plt.xlabel('Exposure Condition and Mitochondrial Parameter', fontsize=12)
plt.ylabel('Exposure Condition', fontsize=12)

# Save the heatmap as a PNG file
#plt.savefig(r"D:\Mitochondria Morphology Analysis\clustered_PerCell_Dunn_Test_w_Bonferroni_FA_07-16-24.png", bbox_inches='tight', dpi=2400)
plt.show()